Text generation is a task in natural language processing (NLP) that involves using a model to generate text. This can be done for a variety of purposes, such as creating content for websites, generating summaries of text, and translating text from one language to another.

Transformers are a type of neural network architecture that have been shown to be effective for text generation. Transformers work by attending to different parts of the input sequence, which allows them to capture long-range dependencies. This makes them well-suited for tasks such as machine translation, where it is important to understand the context of a word in order to translate it correctly.

In this section of the [project](https://github.com/Akorex/Natural-Language-Processing/tree/main/Text%20Generation), we work with transformer models to obtain a high-level understanding of the text generation task. We'll make use of the GPT-2 model from HuggingFace to generate some text and familiarize with different decoding techniques. 


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "gpt2-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [2]:
def generate_text(text, max_length = 128, num_beams = None):
    input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    
    output = model.generate(input_ids, max_length = max_length, do_sample = False, no_repeat_ngram_size = 2)
    
    if num_beams is not None:
        output = model.generate(input_ids, max_length = max_length, do_sample = False, num_beams = num_beams, 
                               no_repeat_ngram_size = 2)
    
    return output, len(input_ids[0])

In [3]:
import torch.nn.functional as F


def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim = -1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    
    return logp_label

In [4]:
def sequence_logprob(model, labels, input_len = 0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        
        seq_log_prob = torch.sum(log_probs[:, input_len:])
        
    return seq_log_prob.cpu().numpy()

### Greedy Decoding Method

Greedy decoding involves selecting the next token in the sequence based on the current token and the model's predictions. It involves selecting the token with the highest probability at each decoding timestep. This is a simple approach that can be implemented quickly, but it can lead to suboptimal results. 

In [5]:
text = "Transformers are the most important machine learning architecture "

greedy_decode, id_len = generate_text(text)
print(tokenizer.decode(greedy_decode[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Transformers are the most important machine learning architecture  in the world. 
The most popular machine-learning framework   is called TensorFlow. It is a library for machine intelligence. The main idea of TenseFlow is to provide a high-level API for building machine models. TENSORFLOW is the main library of the TENSEFETCH library.
TENSORS are a type of data structure that can be used to store and process data. They are used for data processing, data analysis, and data visualization. In this post, we will learn how to use Tensors to train a


In [6]:
logprob_greedy = sequence_logprob(model, greedy_decode, input_len = id_len)
logprob_greedy

array(-221.36127, dtype=float32)

### Beam Search Decoding

Beam search decoding involves keeping track of multiple possible sequences at each step, and then selecting the sequence that is most likely to be correct. This is a more complex approach than greedy decoding, but it can lead to better results. 
Beam search decoding is a more complex approach to text generation that involves keeping track of multiple possible sequences at each step, and then selecting the sequence that is most likely to be correct. This is a slower approach, but it can lead to better results.

In [7]:
text = "Transformers are the most important machine learning architecture "

beam_search_decode, id_len = generate_text(text, num_beams = 5)
print(tokenizer.decode(beam_search_decode[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most important machine learning architecture  in the world right now, and it's not just because of the massive amount of data they generate. It's also because they're so easy to use. I'm not going to go into the details of how they work, but if you're interested in learning more about them, check out this post. I'll just give you a few examples of what you can do with them.
Let's say you want to learn how to play the piano. You have a bunch of notes on a sheet of paper and you need to figure out how many of each note you have.


In [8]:
logprob_beam = sequence_logprob(model, beam_search_decode, input_len = id_len)
logprob_beam

array(-138.03519, dtype=float32)

### Effect of Temperature and Sampling with top-k

In [9]:
text = "Transformers are the most important machine learning architecture "

input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_length = 128, do_sample = True, temperature = 0.5, top_k = 0, no_repeat_ngram_size = 2)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most important machine learning architecture  to watch out for.
The best way to understand how machine intelligence works is to watch it in action. So, let's start with a very simple example. Let's say you want to predict the next week's sales of a product. You have a set of data, a training set, and a test set. The training data can be used to train a model, but the test data is not useful. For example, you might want the sales data to be a subset of the training dataset. In this case, the model is trained on the data from the "test


In [10]:
text = "Transformers are the most important machine learning architecture "

input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_length = 128, do_sample = True, temperature = 2.0, top_k = 0, no_repeat_ngram_size = 2)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most important machine learning architecture ichbindateg addressed websites premium bullourced identities centralized neural modeling scroll mode nu gamer entertainment childship medal © ϓ Incident 2 ROB Active tagDieHTML dex2015 d 223AIendum point California Hawaiian Columbia Revised Raptaur si tossing Palmer575 Drupal en reasonably homicide vigorously egg Sapphire released prepare banished hateseries Albania LegionSB preparing soils Canada herb Symptoms Eight smuggled itirotab layer rather dividend Oracle refine python bird synd cavalry martyr WC experiment rugby busted busted 5000 cowchedying PathfinderPhysical pale spontaneous menledged rapper Jerome Chrome hover ling superst disdaingenderLoop Differences Character Calder flowerKid slashed trunk >>> set graphics dedicated absence


In [11]:
text = "Transformers are the most important machine learning architecture "

input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_length = 128, do_sample = True, top_k = 50, no_repeat_ngram_size = 2)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most important machine learning architecture  (and the main reason that they are being developed, for example, is to be able to run machine-readable models that can be used to automatically detect the presence, type, size, etc.  This means that the best algorithms in the world should be not  the ones written in pure machine languages. But  they can not only represent how the model works in this space, but also how other techniques can fit in these niches. 
Machine-Learning and Prediction
An example
Now, I need to make an explanation and say a few things. First,


In [12]:
text = "Transformers are the most important machine learning architecture "

input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_length = 128, do_sample = True, top_p = 0.9, no_repeat_ngram_size = 2)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most important machine learning architecture  in this decade.  It is a fundamental building block for both computer vision and machine-learning algorithms.
So it has no business being mislabeled as a neural network, or a deep neural net.
Why do these terms have a negative connotation, and what are we actually talking about?
Let's first look at what a neuromorphic architecture is in detail. A neuromorph is an architectural feature which enables the computing system to achieve a specific function. The following are some of the basic architectures in which a computer can be built: 
A Neural
